In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from src.dataset import ProteinDataset
from src.utils import train_model, test_model

/tmp/ipykernel_6356/957630309.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
csv_file = 'data/strict.csv'
chemical_shifts_df = pd.read_csv(csv_file)

In [4]:
test_ids = []
with open("pdb_matched/final_test_ids.txt", "r") as f:
    for line in f:
        test_ids.append(line.strip())

In [5]:
train_set = chemical_shifts_df[~chemical_shifts_df['ID'].isin(test_ids)]
test_set = chemical_shifts_df[chemical_shifts_df['ID'].isin(test_ids)]

In [6]:
train_set

,Unnamed: 0,ID,entryID,stID,entity_assemID,entityID,seq_index,seq,k,zscores,pscores,C,CA,CB,HA,H,N,HB
0,0,19347_1_1_1,19347,1,1,1,1,M,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,19347_1_1_1,19347,1,1,1,2,D,8,2.4350,0.6780,176.004,54.180,41.529,NaN,NaN,NaN,NaN
2,0,19347_1_1_1,19347,1,1,1,3,V,13,1.9193,0.7580,175.968,62.650,32.596,NaN,8.256,120.365,NaN
3,0,19347_1_1_1,19347,1,1,1,4,F,15,1.5919,0.7857,175.889,57.960,39.367,NaN,8.364,123.495,NaN
4,0,19347_1_1_1,19347,1,1,1,5,M,15,-0.8221,0.9041,175.992,55.289,32.629,NaN,8.242,122.277,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217488,1909,15582_1_1_1,15582,1,1,1,11,S,15,12.8633,0.0323,NaN,61.850,63.140,4.28,8.010,NaN,4.025
217489,1909,15582_1_1_1,15582,1,1,1,12,L,13,11.8975,0.0365,NaN,56.990,43.040,4.30,8.120,NaN,1.750
217490,1909,15582_1_1_1,15582,1,1,1,13,T,11,9.8587,0.0980,NaN,NaN,NaN,4.43,7.980,NaN,4.360
217491,1909,15582_1_1_1,15582,1,1,1,14,G,6,6.6457,0.1602,NaN,45.630,NaN,4.06,8.110,NaN,NaN


In [19]:
target_columns = ['N']

train_set.dropna(inplace=True, subset=target_columns)
test_set.dropna(inplace=True, subset=target_columns)

# first baseline - average value of the target column
baseline = train_set[target_columns].mean()

# second baseline - average value of the target column for each amino acid
baseline2 = train_set.groupby('seq')[target_columns].mean()

/tmp/ipykernel_6356/3985583152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set.dropna(inplace=True, subset=target_columns)
/tmp/ipykernel_6356/3985583152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.dropna(inplace=True, subset=target_columns)


In [21]:
# evaluate baselines
# first baseline RMSE on test
import numpy as np
baseline_rmse = np.mean((test_set[target_columns].to_numpy() - baseline.to_numpy()) ** 2) ** 0.5


In [23]:
# second baseline RMSE on test
baseline2_rmse = np.mean((test_set[target_columns].to_numpy() - baseline2.loc[test_set['seq']].to_numpy()) ** 2) ** 0.5

In [27]:
# bootstrap first baseline to estimate 0.95 confidence interval for RMSE
errors = (test_set[target_columns].to_numpy() - baseline.to_numpy()) ** 2
# add results to the df
test_set[f'baseline_{target_columns[0]}'] = baseline.to_numpy()


bootstrap_rmse = []
for i in range(1000):
    bootstrap_sample = np.random.choice(errors.flatten(), int(errors.size * 0.5))
    bootstrap_rmse.append(np.mean(bootstrap_sample) ** 0.5)
    
bootstrap_rmse = np.array(bootstrap_rmse)
bootstrap_rmse.sort()
bootstrap_rmse[25], bootstrap_rmse[975]

(4.978042129484184, 5.209119229942133)

In [28]:
# bootstrap second baseline to estimate 0.95 confidence interval for RMSE
errors = (test_set[target_columns].to_numpy() - baseline2.loc[test_set['seq']].to_numpy()) ** 2
bootstrap_rmse = []
for i in range(1000):
    bootstrap_sample = np.random.choice(errors.flatten(), int(errors.size * 0.5))
    bootstrap_rmse.append(np.mean(bootstrap_sample) ** 0.5)

bootstrap_rmse = np.array(bootstrap_rmse)
bootstrap_rmse.sort()
bootstrap_rmse[25], bootstrap_rmse[975]

(3.732239718790797, 3.9027436637998196)